# Upload datasets to elastisearch

#### Bringing mapped baseline and movie scripts into a valid json format and push them to elasticsearch running on server.

1. Map baseline to movie scripts

In [1]:
%%bash
. ~/.bashrc
python3 -m pip install -r ../requirements.txt
# Install necessary packages

bash: line 1: /Users/I518302/.bashrc: No such file or directory


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2936 sha256=ebd30c827ad73f193165013614802743b1b723460b672d573c3d0d9b6cce1686
  Stored in directory: /Users/I518302/Library/Caches/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


In [3]:
import spacy

en = spacy.load('en_core_web_sm', disable=["ner"])

In [4]:


import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(en.Defaults.stop_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/I518302/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/I518302/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/I518302/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/I518302/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/I518302/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
len(stopwords)

505

In [12]:
def good_token(token):
    if isinstance(token, str) and len(token) > 0 and token not in stopwords:
        return True
    else:
        return False

def remove_stopwords(text):
    print(text)
    output = [i for i in text if good_token(i)]
    return output

In [13]:
# index key for further queries
index = "raw_movies"
# Elasticsearch credentials
user = "elasticuser"
pw = "blaubarsch5"
host = "https://app.leon-remke.jakob-hennighausen.melkonyan-davit.de/api/"

In [14]:
# Imorts
import os

from elasticsearch7 import Elasticsearch
import numpy as np
import spacy
import matplotlib
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from collections import Counter
from profanity_filter import ProfanityFilter

In [19]:

es = Elasticsearch(
    hosts=host,
    http_auth=(user, pw)
)

es.info()

/Users/I518302/opt/anaconda3/lib/python3.9/site-packages/elasticsearch7/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'ubuntu-elastic-search',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'C3IT-gleTX-yJIPo-o8IxA',
 'version': {'number': '7.17.8',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '120eabe1c8a0cb2ae87cffc109a5b65d213e9df1',
  'build_date': '2022-12-02T17:33:09.727072865Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [23]:
def get_string_from_file(file):
    with open('../script/' + file, 'r') as f:
        return f.read()

/var/folders/7l/m32y4qxj0412mg3kymv4s8680000gn/T/ipykernel_27296/3926926587.py:1: ConversionWarning: Some errors were detected !
    Line #93 (got 6 columns instead of 7)
    Line #106 (got 2 columns instead of 7)
    Line #127 (got 4 columns instead of 7)
    Line #139 (got 2 columns instead of 7)
    Line #158 (got 2 columns instead of 7)
    Line #191 (got 2 columns instead of 7)
    Line #225 (got 2 columns instead of 7)
    Line #251 (got 2 columns instead of 7)
    Line #271 (got 4 columns instead of 7)
    Line #421 (got 2 columns instead of 7)
    Line #432 (got 6 columns instead of 7)
    Line #453 (got 2 columns instead of 7)
    Line #462 (got 2 columns instead of 7)
    Line #475 (got 2 columns instead of 7)
    Line #477 (got 6 columns instead of 7)
    Line #608 (got 5 columns instead of 7)
    Line #612 (got 2 columns instead of 7)
    Line #702 (got 2 columns instead of 7)
    Line #706 (got 2 columns instead of 7)
    Line #794 (got 2 columns instead of 7)
    Line #89

In [29]:
from os import listdir
from os.path import isfile, join

def get_files_from_dir(dir):
    onlyfiles = [f for f in listdir(dir) if isfile(join(dir, f))]
    return onlyfiles

In [35]:
output_file = open("elastic_search_init.json", "w")

for idx, file in enumerate(get_files_from_dir('../script')):
    script = get_string_from_file(file)
    script = script.replace("'", " ").replace('"', ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\b', ' ').replace('\\', ' ')
    imdb_id = file.split('.')[0]
    where_mapping = np.where(pg_mapping[:,0] == imdb_id)
    if len(where_mapping[0]) == 0:
        continue
    pg_rating = pg_mapping[where_mapping][0][1]
    nudity = pg_mapping[where_mapping][0][2]
    violence = pg_mapping[where_mapping][0][3]
    profanity = pg_mapping[where_mapping][0][4]
    alcohol = pg_mapping[where_mapping][0][5]
    frightening = pg_mapping[where_mapping][0][6]
    value_str = '{"imdb_id": "' + imdb_id + '", "movie_script": "' + script + '", "age_rating": "' + str(pg_rating) + '", "nudity": "' + str(nudity) + '", "violence": "' + str(violence) + '", "profanity": "' + str(profanity) + '", "alcohol": "' + str(alcohol) + '", "frightening": "' + str(frightening) + '"}'
    output_file.write('{"index":{"_index": "' + str(index) + '", "_type":"_doc", "_id": "' + str(idx + 1) + '"}}' + '\n')
    output_file.write(value_str + '\n')
    # if idx == 400:
    #     break
    if idx == len(get_files_from_dir('../script'))-1:
        print('Finished', idx)
output_file.close()
    # Not working yet
    # es.index(index='movies', document={"imdb_id": file, "script": script, "age_rating": pg_rating})
    # es.index(index=index, document={"imdb_id": file, "script": script, "age_rating": pg_rating})

Finished 478


In [ ]:
%%bash
. ~/.bashrc
curl -u elasticuser:blaubarsch5 -H "Content-Type:application/x-ndjson" -XPOST https://app.leon-remke.jakob-hennighausen.melkonyan-davit.de/api/_bulk?pretty --data-binary "@elastic_search_init.json"



bash: line 1: /Users/leonremke/.bashrc: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


{
  "took" : 13897,
  "errors" : false,
  "items" : [
    {
      "index" : {
        "_index" : "raw_movies",
        "_type" : "_doc",
        "_id" : "1",
        "_version" : 20,
        "result" : "updated",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 2044,
        "_primary_term" : 2,
        "status" : 200
      }
    },
    {
      "index" : {
        "_index" : "raw_movies",
        "_type" : "_doc",
        "_id" : "2",
        "_version" : 19,
        "result" : "updated",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 2045,
        "_primary_term" : 2,
        "status" : 200
      }
    },
    {
      "index" : {
        "_index" : "raw_movies",
        "_type" : "_doc",
        "_id" : "3",
        "_version" : 17,
        "result" : "updated",
        "_shards" : {
          "total" : 2,
          "successful" :

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        "result" : "updated",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 2389,
        "_primary_term" : 2,
        "status" : 200
      }
    },
    {
      "index" : {
        "_index" : "raw_movies",
        "_type" : "_doc",
        "_id" : "347",
        "_version" : 3,
        "result" : "updated",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 2390,
        "_primary_term" : 2,
        "status" : 200
      }
    },
    {
      "index" : {
        "_index" : "raw_movies",
        "_type" : "_doc",
        "_id" : "348",
        "_version" : 5,
        "result" : "updated",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 2391,
        "_primary_term" : 2,
        "status" : 200
      }
    },
    {
      "index" : {
        "_index" : "raw_mo

100 96.5M  100  168k  100 96.3M   4394  2515k  0:00:39  0:00:39 --:--:-- 40339


In [6]:
mapping = es.indices.get_mapping(index=index)
mapping

{'raw_movies': {'mappings': {'properties': {'age_rating': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'imdb_id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'movie_script': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}}

In [7]:
es.search(index=index, query={
        "match_all": {}
     })

{'took': 59,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 479, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'raw_movies',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0,
    '_ignored': ['movie_script.keyword'],
    '_source': {'imdb_id': 'tt0113101',
     'movie_script': '                                        FOUR ROOMS                                         Screenplay by                                        Allison Anders                                      Alexandre Rockwell                                       Robert Rodriguez                                      Quentin Tarantino                                   MAIN TITLES                 As presentation credits begin, we hear Johnny Cash s  Home                 of the Blues.  Then we see Allison s name, under it Alex s,                 under that Robert s, under that Quentin s, then under that                 the title logo